In [1]:
from mesh import Mesh

In [2]:
#minimal
#filename, quality_index = "Data/simplicial_donut_3.ply", 2
#small
#filename, quality_index = "Data/simplicial_donut_3_deform.ply", 2
filename, quality_index = "Data/31_r1.00_n4_v256.volume.ply", 3
#filename, quality_index = "Data/31_mean_curvature.ply", 7
#medium
#filename, quality_index = "Data/HS_0109_HeiCuBeDa_GigaMesh.ply", 3
#filename, quality_index = "Data/HS_0109_HeiCuBeDa_GigaMesh_mean_curvature.ply", 7
#large:
#filename, quality_index = "Data/HS_190_HeiCuBeDa_GigaMesh.ply", 3

# Mesh class:
The Mesh class can be initialized using **Mesh()** . It has the following variables and functions:

## Variables:
- **Mesh.filename** filename as string. saved without .ply, but with folder structure

- **Mesh.Vertices** Vertices stored as a dictionary of Vertex objects, keys are Vertex indices

- **Mesh.Edges** Edges stored as a dictionary of Edge objects, keys are numbered

- **Mesh.Faces** Faces stored as a dictionary of Face objects, keys are numbered

- **Mesh.MorseComplex** if not calculated yet: None, otherwise a MorseComplex object with MorseComplex.persistence = 0

- **Mesh.reducedMorseComplexes** a dictionary of MorseComplex objects, that have been reduced by a persistence parameter. The persistence parameter also defines the key of each MorseComplex object

- **Mesh.MorseCells** a dictionary of MorseCells dictionaries. the key for each MorseCell dictionary is given by the persistence of the MorseComplex, these Cells were taken from. Each MorseCell dictionary consits of key, value pairs, where the key gives the label of a cell and the value is a set of vertices that make up this cell.

## Functions:
- **Mesh.load_mesh_ply(filename, quality_index)** loads a .ply file into the Mesh class and takes the value given at quality_index position as the scalar function on the vertices

- **Mesh.info()** prints out Mesh info

- **Mesh.ProcessLowerStars()** calculates the combinatorial gradient and critical simplices; **required for ExtractMorseComplex**

- **Mesh.only_return_ExtractMorseComplex** return MorseComplex without storing it in the Mesh class

- **Mesh.ExtractMorseComplex** calculate MorseComplex and store it in the Mesh class

- **Mesh.only_return_ReducedMorseComplex(persistence)** reduce the MorseComplex based on the given persistence parameter, but dont store it in the Mesh class

- **Mesh.ReducedMorseComplex(persistence)** reduce the MorseComplex based on the given persistence parameter, and store it in the Mesh class

- **Mesh.plot_MorseComplex(MorseComplex, filename, path_color=[255,0,255])** plots the critical simplices of a MorseComplex in red (vertex), green (edge) and blue (face) and the according paths connecting them in magenta. Only writes a ply file containing colored points, so should be used as an overlay for the original mesh

- **Mesh.ExtractMorseCells(MorseComplex)** takes a MorseComplex and gets all connected cells individually that are enclosed by the lines between critical simplices

- **Mesh.plot_MorseCells(persistence, filename)** plots the connected components in each Cell in the same color. Only writes a ply file containing colored points, so should be used as an overlay for the original mesh

In [3]:
Mesh = Mesh()
Mesh.load_mesh_ply(filename, quality_index, inverted=False)
Mesh.info()
Mesh.ProcessLowerStars()
Mesh.ExtractMorseComplex()
Mesh.MorseComplex.info()

Time read data file: 6.0949537
Time read and prepare data: 22.3389742
+-------------------------------------------------------
| Mesh Info
+-------------------------------------------------------
| Filename:  Data/31_r1.00_n4_v256.volume
| Morse function values range:  [-0.5752798, 0.6062798]
+-------------------------------------------------------
| Number of Vertices:  146174
| Number of Edges:  438516
| Number of Faces:  292344
+-------------------------------------------------------
| Euler characteristic:  2
+-------------------------------------------------------
Time ProcessLowerStar: 6.563777600000002
Time ExtractMorseComplex and Separatrices: 3.8155110999999984
+-------------------------------------------------------
| MorseComplex Info
+-------------------------------------------------------
| Filename:  Data/31_r1.00_n4_v256.volume
| Persistence of this Complex:  0
+-------------------------------------------------------
| Number of Vertices:  8878
| Number of Edges:  21092


In [4]:
red_MSC = Mesh.ReducedMorseComplex(2)

Time cancel critical points with  2  persistence:  4.042885200000001
Persistence was higher than the range of function values,
therefore this complex is maximally reduced and can be used for salient edge extraction.


In [5]:
#Mesh.plot_salient_edge_histogram(100, log=True)

In [6]:
#red_MSC.Separatrices.sort(key=lambda x: x[0])
#for pers,sep in red_MSC.Separatrices:
#    print(pers)


In [7]:
#thr = [1,2,5,8,10,15,20,25,100,600,900]
#min_thresh = 10
#max_thresh = 7
#for thresh in thr:
#    Mesh.plot_improved_salient_edge(str(red_MSC.filename)+"_improved_salient_edge_justSum"+str(thresh), 
#                                    thresh, min_thresh, max_thresh)
#    Mesh.plot_salient_edge(str(red_MSC.filename)+"_salient_edge_justSum"+str(thresh), thresh)

In [8]:
# Calculate reduced Complexes for different persistence levels:

persistences = [0.015,0.2]
for thresh in persistences:
    Mesh.ReducedMorseComplex(thresh)
    
# plot MorseComplexes, calculate MorseCells and plot MorseCells
for persistence in persistences:
    reduced_complex = Mesh.reducedMorseComplexes[persistence]
    #reduced_complex.info()
    #Mesh.plot_MorseComplex(reduced_complex, str(reduced_complex.filename)+"_mean_inverted"+str(persistence))
    
    Mesh.ExtractMorseCells(reduced_complex)
    Mesh.write_MorseCellLabels(persistence, str(reduced_complex.filename)+"_error_removed_test_no_bd"+str(persistence))
    #Mesh.plot_MorseCells(persistence, str(reduced_complex.filename)+"_error_removed_test_no_bd"+str(persistence))


Time cancel critical points with  0.015  persistence:  3.4391226999999986
Time cancel critical points with  0.2  persistence:  2.858816300000001
Time get MorseCells:  1.0004849000000036
Time writing label txt file: 0.14200629999999848
Time get MorseCells:  1.0601343999999955
Time writing label txt file: 0.12627970000000488


In [9]:
graph = Mesh.GetConnectivityGraph(0.2)

Nb of not found labels:  112
Time get weighted ConnectivityGraph:  0.14261020000000002


In [10]:
for key, val in graph.conncomps.items():
    if key==7:
        print("label 7 connected to:")
        for ke, wei in val.items():
            print(ke, "with weight", wei)

label 7 connected to:
0 with weight -0.2400329311656234
4 with weight 0.019243094846873792
5 with weight 0.035927737010312606
9 with weight 0.042309492410043
23 with weight 0.060242882523328135
22 with weight 0.050435352291508924
10 with weight -0.016611210988391495
81 with weight 0.023889683187007904
35 with weight -0.19884283188730478
115 with weight 0.023391450755298138
78 with weight 0.024878489186189005
31 with weight 0.02442047791555524
59 with weight 0.04235781170427799
24 with weight 0.10496249215974085
129 with weight 0.07962565636262298
116 with weight 0.070761495269835
130 with weight 0.057187542668543756
131 with weight 0.029097028821706773
27 with weight 0.05227638501673937
17 with weight 0.027777397442326945
111 with weight 0.043377814426397286
126 with weight 0.07715565711259842
132 with weight 0.044317695001761116
57 with weight 0.05727239378861019
112 with weight 0.05092801107093692
125 with weight 0.03477180376648903
113 with weight 0.03448134753853083
56 with weight 

In [11]:
print(len(graph.conncomps.keys()))

138
